In [1]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
                         summarize,
                         poly)
from sklearn.model_selection import train_test_split
from functools import partial
from sklearn.model_selection import \
     (cross_validate,
      KFold,
      ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

In [2]:
Auto = load_data('Auto')
Auto_train, Auto_valid = train_test_split(Auto,
                                         test_size=196,
                                         random_state=0)
# random seed set as 0, so that the same randomness will be used next time we execute the code and the results will be consistent

In [3]:
hp_mm = MS(['horsepower'])
X_train = hp_mm.fit_transform(Auto_train)
y_train = Auto_train['mpg']
model = sm.OLS(y_train, X_train)
results = model.fit()

In [4]:
X_valid = hp_mm.transform(Auto_valid)
y_valid = Auto_valid['mpg']
valid_pred = results.predict(X_valid)
np.mean((y_valid - valid_pred) ** 2)

np.float64(23.616617069669886)

In [5]:
# validation set MSE is 23.61%

In [6]:
def evalMSE(terms, response, train, test):
    
    mm = MS(terms)
    X_train = mm.fit_transform(train)
    y_train = train[response]

    X_test = mm.transform(test)
    y_test = test[response]

    results = sm.OLS(y_train, X_train).fit()
    test_pred = results.predict(X_test)

    return np.mean((y_test - test_pred)**2)

In [7]:
MSE = np.zeros(3)
for idx, degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)],
                       'mpg',
                       Auto_train,
                       Auto_valid)
MSE

array([23.61661707, 18.76303135, 18.79694163])

In [8]:
"""
the results will be different if we choose different splits or different seed number of the randomness
"""

'\nthe results will be different if we choose different splits or different seed number of the randomness\n'

In [9]:
# cross validation

In [10]:
# sklearn_sm() wraps the scikit's cross-validation tools over the statsmodel's fit
# additional args : model_str -> model formula & model_args -> dictionary of additional args used to fit the model

hp_model = sklearn_sm(sm.OLS, MS(['horsepower']))
X, Y = Auto.drop(columns = ['mpg']), Auto['mpg']
cv_results = cross_validate(hp_model, X, Y, cv = Auto.shape[0])
cv_err = np.mean(cv_results['test_score'])
cv_err

np.float64(24.231513517929233)

In [ ]:
"""
by default the cross validation thats run is 5 fold
if n is the number of observations, LOOCV is run
if the Y is binary or categorical, Stratified K fold is used

cross_validate returns a dictionary of results which includes test_score, train_score, fit_time, score_time
"""

In [12]:
cv_error = np.zeros(5)
H = np.array(Auto['horsepower'])
M = sklearn_sm(sm.OLS)
for i, d in enumerate(range(1,6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M,
                          X,
                          Y,
                          cv=Auto.shape[0])
    cv_error[i] = np.mean(M_CV['test_score'])
cv_error

array([24.23151352, 19.24821312, 19.33498406, 19.42443032, 19.03323829])

In [ ]:
"""
outer executes the function on each pair of the 2 matrices
in the above code, H is an array of horsepower values. 
in each loop, the X becomes a matrix of 
1,H0
1,H0,H1 
1,H0,H1,H2... so on. 
to be interpreted as mpg is polynomial function of horsepower
"""

In [ ]:
# K fold cross validation

In [14]:
cv_error = np.zeros(5)
cv = KFold(n_splits = 10,
           shuffle = True,
           random_state = 0) # use same splits for each degree
for i, d in enumerate(range(1,6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M,
                          X,
                          Y,
                          cv = cv)
    cv_error[i] = np.mean(M_CV['test_score'])
cv_error

array([24.20766449, 19.18533142, 19.27626666, 19.47848403, 19.13721179])

In [15]:
# cross validate supports ShuffleSplit as well
validation = ShuffleSplit(n_splits=1,
                          test_size=196,
                          random_state=0)
results = cross_validate(hp_model,
                         Auto.drop(['mpg'], axis=1),
                         Auto['mpg'],
                         cv=validation);
results['test_score']

array([23.61661707])

In [ ]:
"""
randomly shuffles the dataset and splits it into training and test sets over multiple iterations. 
Unlike K-Fold, it allows for independent sampling, meaning some data points may be repeated in the test sets across iterations, 
while allowing control over the number of iterations. 
"""